<a href="https://colab.research.google.com/github/wisrovi/RedesNeuronales/blob/master/AprendizajePorRefuerzo-PongDeterministic2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teoría

## Objetivo: 
Repasar los conceptos vistos en clase.

La puntuación de este bloque es de 4 puntos sobre la nota final.

**Define brevemente qué es el aprendizaje por refuerzo. ¿Qué diferencias hay entre aprendizaje supervisado, no supervisado y por refuerzo?**

**Define con tus palabras los conceptos de Entorno, Agente, Recompensa, Estado y Observación.**

**Dependiendo del algoritmo de aprendizaje por refuerzo que se use, ¿qué clasificaciones podemos encontrar? Coméntalas brevemente.**




**Lista tres diferencias entre los algoritmos de DQN y Policy Gradient**

# Práctica

Objetivo: Implementar una solución, usando keras-rl y basada en el algoritmo de DQN visto en clase, para que un agente aprenda una estrategia ganadora en el juego del Pong.

La puntuación de este bloque es de 6 puntos sobre la nota final.

El entorno sobre el que trabajaremos será _PongDeterministic-v0_ y el algoritmo que usaremos será _DQN_.

Para evaluar cómo lo está haciendo el agente, la recompensa en el _Pong_ oscila, aproximadamente, en el rango de valores **[-20, 20]**. La estrategia óptima de un agente estaría alrededor de una media de recompensa de 20.

- **NOTA IMPORTANTE**: Si el agente no llegara a aprender una estrategia ganadora, responder sobre la mejor estrategia obtenida.

In [54]:
#para instalar la insterfaz grafica
!apt-get install python-opengl -y
!apt install xvfb -y

#para rendereizar datos
!pip install pyvirtualdisplay
!pip install piglet

#para activar la interfaz visual para entrenar el agente
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

#para instalar el visual de atari
!pip install “gym[atari]"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.3).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

In [56]:
!pip install keras-rl==0.4.2
!pip install tensorflow==1.13.1
!pip install gym
!pip install gym[atari]
!pip install h5py


#creo son necesarias, pero no se ha comprobado que en su ausencia falle
!pip install jupyter
!pip install torch

In [0]:
from PIL import Image

import numpy as np
import gym

# Para las librerias para la red neuronal
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Conv2D, MaxPooling2D, Dropout
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor

from rl.callbacks import FileLogger, ModelIntervalCheckpoint

In [58]:
ENV_NAME = 'PongDeterministic-v0'

# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
nb_actions = env.action_space.n

# random seed
np.random.seed(123)
env.seed(123)

[123, 151010689]

In [0]:
# Define the input shape to resize the screen
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

# This processor will be similar to the Atari processor
class PongProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        
        img = Image.fromarray(observation)
        # resize and convert to grayscale
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

In [0]:
processor = PongProcessor()

In [0]:
def createModel():
    input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE

    model = Sequential()

    model.add(Permute((2, 3, 1), input_shape=input_shape))
    
    model.add( Convolution2D(8, kernel_size=(16, 16), strides=(5, 5), activation='relu', padding='same'  ) )
    model.add( Convolution2D(16, kernel_size=(16, 16), strides=(5, 5), activation='relu', padding='same'  ) )
    model.add( Convolution2D(32, kernel_size=(8, 8), strides=(4, 4), activation='relu', padding='same'  ) )
    model.add(MaxPooling2D((2, 2),padding='same'))
    model.add( Convolution2D(64, kernel_size=(4, 4), strides=(2, 2), activation='relu', padding='same'  ) )
    model.add( Convolution2D(128, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'  ) )
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25)) #apagar un 25% de manera aleatoria para reducir la cantidad de parametros
    model.add(Dense(nb_actions, activation='linear'))    
    return model

In [62]:
model = createModel()
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_8 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 17, 17, 8)         8200      
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 4, 4, 16)          32784     
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 1, 1, 32)          32800     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 1, 1, 32)          0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 1, 1, 64)          32832     
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 1, 1, 128)        

In [0]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=1000000)

In [0]:
dqn = DQNAgent(
    nb_actions=nb_actions, 
    model=model, 
    policy=policy, 
    memory=memory,
    processor=processor, 
    nb_steps_warmup=50000, 
    target_model_update=10000,
    gamma=.99,
    train_interval=20)

In [0]:
#compilar modelo
optimizer = Adam(lr=1e-3)
dqn.compile(optimizer, metrics=['mae'])

In [0]:
#callback
checkpoint_weights_filename = 'dqn_' + ENV_NAME + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(ENV_NAME)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=50000)]
callbacks += [FileLogger(log_filename, interval=100)]

In [67]:
from google.colab import drive
drive.mount('/content/gdrive')
BASE_FOLDER = '/content/gdrive/My Drive/Master IA/AprendizajePorRefuerzo/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
weights_filename = 'dqn_' + ENV_NAME + '_weights2.h5f'
usarPesosEntrenados = True
if usarPesosEntrenados:
    try:
        dqn.load_weights(BASE_FOLDER + weights_filename)
    except:
        print("No se encontro archivos de pesos")
        pass

scoreOld = -50
entrenar = True
agenteOld = None
if entrenar:
    conteo = 0
    for i in range(5):
    # Training part
        try:
            dqn.fit(env, callbacks=callbacks, nb_steps=200000, log_interval=10000, visualize=False)
        except:
            print("Erro al entrenar")
            pass

        testEval = dqn.test(env, nb_episodes=2, visualize=False).history['episode_reward'][1]

        if scoreOld < testEval:
            scoreOld = testEval
            agenteOld = dqn
            dqn.save_weights(BASE_FOLDER + weights_filename, overwrite=True)
            print("Guardando pesos", scoreOld)
        else:
            print("No hubo mejora.")
            conteo += 1
            if conteo > 3:
                pass
                #break
            #dqn = agenteOld        
else:    
    dqn.load_weights(weights_filename)

Training for 200000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 46s 5ms/step - reward: -0.0205
10 episodes - episode_reward: -20.000 [-21.000, -18.000] - ale.lives: 0.000

Interval 2 (10000 steps performed)
 7807/10000 [======================>.......] - ETA: 9s - reward: -0.0231

In [0]:
dqn.test(env, nb_episodes=2, visualize=False)

In [0]:
#obs0 = env.reset()
#print("initial observation code:", obs0)

plt.imshow(env.render('rgb_array'))
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)